In [154]:
import pandas as pd
from pathlib import Path
import requests
import re

In [155]:
df = pd.read_csv('CRC_KEGG_KOs_cp_support_merge.tsv', sep='\t')
df.columns = ['kegg_ko', 'score', 'name']

In [156]:
for i in df['kegg_ko']:
    ko_file = f'/mnt/lustre/scratch/CBRA/research/projects/metagenomic/gut/metadata/data/raw/kegg/{i}.txt'
    my_file = Path(ko_file)
    if my_file.is_file():
        with open(ko_file) as file:
            for line in file:
                line = line.rstrip()
                if line.startswith('GENES'):
                    break
                if ' RN: ' in line:
                    rn = line.split('RN:')[1]
                    rn = rn.strip()
                    df.loc[df['kegg_ko'] == i, 'kegg_reaction'] = rn
                    
    else:
        print(f'error in {i}')

In [157]:
array = []
for i in df.loc[df['kegg_reaction'].notnull(), 'kegg_reaction']:
    i = i.split(' ')
    array.extend(i)

len(array)

3287

In [158]:
len(set(array))

2314

In [159]:
base = '/mnt/lustre/scratch/CBRA/research/projects/metagenomic/gut/'
for i in array:
    print('\r', i, end='\t')
    
    rn_file = Path(f'{base}metadata/data/raw/kegg/{i}.txt')
    if rn_file.is_file():
        continue
        
    url = 'http://rest.kegg.jp/get/' + i
    r = requests.get(url)
    
    file = open(f"{base}metadata/data/raw/kegg/{i}.txt", "w")
    file.write(r.text)
    file.close()

 R09397	

In [160]:
df['kegg_compound'] = ''
df = df.fillna('NA')

for i in list(set(array)):
    rn_file = f'/mnt/lustre/scratch/CBRA/research/projects/metagenomic/gut/metadata/data/raw/kegg/{i}.txt'
    with open(rn_file, 'r') as file:
        for line in file:
            line = line.rstrip()
            if line.startswith('EQUATION'):
                comp = re.findall(r'[CG]\d+', line)
                comp = ' '.join(comp)
                df.loc[df['kegg_reaction'].str.contains(i), 'kegg_compound'] = df['kegg_compound'] + ' ' + comp



In [161]:
kegg_comp = df.copy()
kegg_comp['kegg_compound'] = kegg_comp['kegg_compound'].str.strip().str.split().apply(lambda x: list(set(x)))
kegg_comp = kegg_comp.explode('kegg_compound').groupby('kegg_compound').sum().sort_values('score', ascending=False).head(200)

for i in kegg_comp.index:
    print('\r', i, end='\t')
    
    c_file = Path(f'{base}metadata/data/raw/kegg/{i}.txt')
    if c_file.is_file():
        continue
        
    url = 'http://rest.kegg.jp/get/' + i
    r = requests.get(url)
    
    file = open(c_file, "w")
    file.write(r.text)
    file.close()
    
for i in kegg_comp.index:
    c_file = f'{base}metadata/data/raw/kegg/{i}.txt'
    with open(c_file, 'r') as file:
        for line in file:
            line = line.rstrip()
            if line.startswith('NAME'):
                name = line.replace(';', '').split()[1]
                kegg_comp.loc[i, 'name'] = name
kegg_comp.head(20)

 C05761	

,score,name
kegg_compound,,
C00001,74.469208,H2O
C00080,44.892296,H+
C00002,35.383378,ATP
C00009,27.225732,Orthophosphate
C00003,26.484619,NAD+
C00004,25.293150,NADH
C00008,22.438070,ADP
C00013,19.339948,Diphosphate
C00011,16.325968,CO2


In [162]:
kegg_rn = df.copy()
kegg_rn['kegg_reaction'] = kegg_rn['kegg_reaction'].str.strip().str.split().apply(lambda x: list(set(x)))
kegg_rn = kegg_rn.explode('kegg_reaction').groupby('kegg_reaction').sum().sort_values('score', ascending=False).head(200)

for i in kegg_rn.index:
    print('\r', i, end='\t')
    
    rn_file = Path(f'{base}metadata/data/raw/kegg/{i}.txt')
    if rn_file.is_file():
        continue
        
    url = 'http://rest.kegg.jp/get/' + i
    r = requests.get(url)
    
    file = open(rn_file, "w")
    file.write(r.text)
    file.close()
    
for i in kegg_rn.index:
    rn_file = f'{base}metadata/data/raw/kegg/{i}.txt'
    with open(rn_file, 'r') as file:
        for line in file:
            line = line.rstrip()
            if line.startswith('NAME'):
                name = line.replace(';', '').split()[1]
                kegg_rn.loc[i, 'name'] = name
kegg_rn.head(20)

 R04927	

,score,name
kegg_reaction,,
NA,368.683143,NaN
R11945,2.845164,NADH:ubiquinone
R00262,1.505437,L-threo-3-methylaspartate
R02111,1.387899,"1,4-alpha-D-Glucan:orthophosphate"
R01715,1.378665,Chorismate
R03696,1.349319,L-threo-3-Methylaspartate
R06927,1.196699,(2-naphthyl)methanol:NAD+
R00754,1.196699,ethanol:NAD+
R00623,1.196699,primary_alcohol:NAD+


In [163]:
kegg_comp

,score,name
kegg_compound,,
C00001,74.469208,H2O
C00080,44.892296,H+
C00002,35.383378,ATP
C00009,27.225732,Orthophosphate
C00003,26.484619,NAD+
...,...,...
C04574,1.114136,"di-trans,poly-cis-Undecaprenyl"
C01172,1.111377,beta-D-Glucose
C00857,1.107547,Deamino-NAD+


In [ ]:
print(df.head(20).to_markdown())

In [76]:
# glycine HMDB0000123
df.loc[df['kegg_compound'].str.contains('C00037'), 'score'].sum()

4.823568715305929

In [77]:
# L-Valine HMDB0000883
df.loc[df['kegg_compound'].str.contains('C00183'), 'score'].sum()

0.693087898339176

In [78]:
# L-Alanine HMDB0000161
df.loc[df['kegg_compound'].str.contains('C00041'), 'score'].sum()

3.2273935100960536

In [79]:
# L-Lactic acid HMDB0000190 C00186
df.loc[df['kegg_compound'].str.contains('C00186'), 'score'].sum()

0.527994435621725

In [80]:
# L-Phenylalanine HMDB0000159 C00079
df.loc[df['kegg_compound'].str.contains('C00079'), 'score'].sum()

2.117857375112338

In [81]:
# L-Proline HMDB0000162
df.loc[df['kegg_compound'].str.contains('C00148'), 'score'].sum()

1.434113834907183

In [82]:
# L-Leucine HMDB0000687
df.loc[df['kegg_compound'].str.contains('C00123'), 'score'].sum()

0.300635423353933

In [83]:
# L-Glutamic acid  HMDB0000148
df.loc[df['kegg_compound'].str.contains('C00025'), 'score'].sum()

13.325349025335168

In [20]:
# Taurine HMDB0000251
df.loc[df['kegg_compound'].str.contains('C00245'), 'score'].sum()

0.910383303132282

In [21]:
# Palmitic acid HMDB0000220
df.loc[df['kegg_compound'].str.contains('C00249'), 'score'].sum()

0.320905344939598

In [22]:
# L-Methionine HMDB0000696
df.loc[df['kegg_compound'].str.contains('C00073'), 'score'].sum()

4.503947811668805

In [23]:
# Glycerol HMDB0000131
df.loc[df['kegg_compound'].str.contains('C00116'), 'score'].sum()

1.1257348575940849

In [24]:
# L-Tyrosine HMDB0000158
df.loc[df['kegg_compound'].str.contains('C00082'), 'score'].sum()

1.718910949113783

In [25]:
# L-Threonine HMDB0000167 C00188
df.loc[df['kegg_compound'].str.contains('C00188'), 'score'].sum()

1.354361220722235

In [26]:
# L-Isoleucine HMDB0000172
df.loc[df['kegg_compound'].str.contains('C00407'), 'score'].sum()

0.387302990567718

In [27]:
# L-Serine HMDB0000187
df.loc[df['kegg_compound'].str.contains('C00065'), 'score'].sum()

2.042185228951431

In [28]:
# L-Aspartic acid HMDB0000191
df.loc[df['kegg_compound'].str.contains('C00049'), 'score'].sum()

3.7527606239752984

In [29]:
# Glucose HMDB0000122
df.loc[df['kegg_compound'].str.contains('C00221'), 'score'].sum()

0.9079622304938311

In [30]:
# L-Lysine HMDB0000182
df.loc[df['kegg_compound'].str.contains('C00047'), 'score'].sum()

1.8451829187478062

In [31]:
# L-Arginine HMDB0000517
df.loc[df['kegg_compound'].str.contains('C00062'), 'score'].sum()

1.818335228953883

In [32]:
# L-Glutamine HMDB0000641
df.loc[df['kegg_compound'].str.contains('C00064'), 'score'].sum()

5.074283424030617

In [33]:
# Choline HMDB0000097
df.loc[df['kegg_compound'].str.contains('C00114'), 'score'].sum()

0.39105533279169796

In [34]:
# L-Asparagine HMDB0000168
df.loc[df['kegg_compound'].str.contains('C00152'), 'score'].sum()

0.776498560163747

In [35]:
# myo-Inositol HMDB0000211
df.loc[df['kegg_compound'].str.contains('C00137'), 'score'].sum()

0.384770591859444

In [36]:
# Succinic acid HMDB0000254
df.loc[df['kegg_compound'].str.contains('C00042'), 'score'].sum()

1.8190666717571156

In [37]:
# L-Tryptophan HMDB0000929
df.loc[df['kegg_compound'].str.contains('C00078'), 'score'].sum()

1.0282954792227428

In [38]:
# Acetic acid HMDB0000042
df.loc[df['kegg_compound'].str.contains('C00033'), 'score'].sum()

4.986949588675744

In [39]:
# Uridine HMDB0000296
df.loc[df['kegg_compound'].str.contains('C00299'), 'score'].sum()

1.3765207677391091

In [40]:
# Myristic acid HMDB0000806
df.loc[df['kegg_compound'].str.contains('C06424'), 'score'].sum()

0.0

In [169]:
#1
compound = ['glycine',
            'Aminoacetic acid',
            'Aminoethanoic acid'
           ]
regex = "|".join(compound)

result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Glycine'
result['HMDB_ID'] = 'HMDB0000123'
result['Frequency'] = 24
# print(result.to_markdown())
r1 = result

In [170]:
compound = ['L-Valine',
            'L-Valin',
            '2-Amino-3-methylbutyric acid',
            'valine'
           ]
regex = "|".join(compound)

result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Valine'
result['HMDB_ID'] = 'HMDB0000883'
result['Frequency'] = 23
# print(result.to_markdown())
r2 = result

In [171]:
compound = ['L-Alanine',
            '(S)-Alanine',
            'L-Alanin',
            '2-Aminopropanoate',
            'l alanine',
            'a-Aminopropionate',
            'a-Aminopropionic acid',
            'Alanine'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Alanine'
result['HMDB_ID'] = 'HMDB0000161'
result['Frequency'] = 22
# print(result.to_markdown())
r3 = result

/home/csanchez/.conda/envs/python3.8/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [172]:
compound = ['L-Lactic acid',
            'L-Lactate',
            '(S)-Lactate',
            '(S)-Lactic acid',
            '(S)-2-Hydroxypropanoic acid',
            'Lactic acid',
            'L-lactic'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Lactic acid'
result['HMDB_ID'] = 'HMDB0000190'
result['Frequency'] = 22
# print(result)
r4 = result

In [173]:
compound = ['L-Phenylalanine',
            '(S)-2-Amino-3-phenylpropionic acid',
            '(S)-alpha-Amino-beta-phenylpropionic acid',
            '3-Phenyl-L-alanine',
            'beta-Phenyl-L-alanine',
            'Phenylalanine',
            'Phenyl-L-alanine'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Phenylalanine'
result['HMDB_ID'] = 'HMDB0000159'
result['Frequency'] = 20
# print(result)
r5 = result

In [174]:
compound = ['L-Proline',
            '(-)-(S)-Proline',
            '(-)-Proline',
            '(2S)-Pyrrolidine-2-carboxylic acid',
            '(S)-2-Carboxypyrrolidine',
            'proline'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Proline'
result['HMDB_ID'] = 'HMDB0000162'
result['Frequency'] = 20
# print(result.to_markdown())
r6 = result

In [175]:
compound = ['L-Leucine',
            '(2S)-2-Amino-4-methylpentanoic acid',
            '(2S)-alpha-2-Amino-4-methylvaleric acid',
            '(2S)-alpha-Leucine',
            'L-Leucin',
            'Leucine',
            'leucin'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Leucine'
result['HMDB_ID'] = 'HMDB0000687'
result['Frequency'] = 20
# print(result.to_markdown())
r7 = result

In [176]:
compound = ['L-Glutamic acid',
            '(S)-2-Aminopentanedioic acid',
            '(S)-Glutamic acid',
            'Glutamate'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Glutamic acid'
result['HMDB_ID'] = 'HMDB0000148'
result['Frequency'] = 17
# print(result.to_markdown())
r8 = result

In [177]:
compound = ['Taurine',
            '2-Aminoethanesulfonic acid',
            '2-Aminoethyl sulfonate',
            'Aminoethylsulfonate',
            'Aminoethylsulphonate'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Taurine'
result['HMDB_ID'] = 'HMDB0000251'
result['Frequency'] = 16
# print(result.to_markdown())
r9 = result

In [178]:
compound = ['Palmitic acid',
            '1-Hexyldecanoic acid',
            'Cetylic acid',
            'Hexadecanoate',
            'Palmitate'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Palmitic acid'
result['HMDB_ID'] = 'HMDB0000220'
result['Frequency'] = 15
# print(result.to_markdown())
r10 = result

In [179]:
compound = ['L-Methionine',
            '(2S)-2-Amino-4-(methylsulfanyl)butanoic acid',
            'METHIONINE',
            'L-Methionin',
            'Methionin'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Methionine'
result['HMDB_ID'] = 'HMDB0000696'
result['Frequency'] = 15
# print(result.to_markdown())
r11 = result

In [180]:
compound = ['Glycerol',
            'Glycerine',
            'Propanetriol',
            'D-Glucuronate'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Glycerol'
result['HMDB_ID'] = 'HMDB0000131'
result['Frequency'] = 14
# print(result.to_markdown())
r12 = result

In [181]:
compound = ['L-Tyrosine',
            'Tyrosine',
            'Benzenepropanoate',
            'Benzenepropanoic acid'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Tyrosine'
result['HMDB_ID'] = 'HMDB0000158'
result['Frequency'] = 14
# print(result.to_markdown())
r13 = result

In [182]:
compound = ['L-Threonine',
            'Threonine',
            'Threonin',
            '2-Amino-3-hydroxybutyric acid'
           ]
regex = "|".join(compound)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Threonine'
result['HMDB_ID'] = 'HMDB0000167'
result['Frequency'] = 14
# print(result.to_markdown())
r14 = result

In [183]:
compound = ['L-Isoleucine',
            'Isoleucine',
            'Alloisoleucine',
            'methylvalerate',
            'Isoleucin'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Isoleucine'
result['HMDB_ID'] = 'HMDB0000172'
result['Frequency'] = 14
# print(result.to_markdown())
r15 = result

In [184]:
compound = ['L-Serine',
            'Serine',
            'L-Serin',
            'Serin',
            'hydroxyalanine'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Serine'
result['HMDB_ID'] = 'HMDB0000187'
result['Frequency'] = 14
# print(result.to_markdown())
r16 = result

In [185]:
compound = ['L-Aspartic acid',
            'Aspartic acid',
            'L-Aspartate',
            'Aspartate',
            'Asparagate'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Aspartic acid'
result['HMDB_ID'] = 'HMDB0000191'
result['Frequency'] = 14
# print(result.to_markdown())
r17 = result

In [186]:
compound = ['Glucose',
            'D-Glucose'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'D-Glucose'
result['HMDB_ID'] = 'HMDB0000122'
result['Frequency'] = 13
# print(result.to_markdown())
r18 = result

In [187]:
compound = ['L-Lysine',
            'Lysine'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Lysine'
result['HMDB_ID'] = 'HMDB0000182'
result['Frequency'] = 12
# print(result.to_markdown())
r19 = result

In [188]:
compound = ['L-Arginine',
            'Arginine'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Arginine'
result['HMDB_ID'] = 'HMDB0000517'
result['Frequency'] = 12
# print(result.to_markdown())
r20 = result

In [189]:
compound = ['L-Glutamine',
            'Glutamine'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Glutamine'
result['HMDB_ID'] = 'HMDB0000641'
result['Frequency'] = 12
# print(result.to_markdown())
r21 = result

In [190]:
compound = ['Choline',
            'Bilineurine',
            'Biocoline'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Choline'
result['HMDB_ID'] = 'HMDB0000097'
result['Frequency'] = 11
# print(result.to_markdown())
r22 = result

In [191]:
compound = ['L-Asparagine',
            'Asparagine',
            'Aspartamate'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Asparagine'
result['HMDB_ID'] = 'HMDB0000168'
result['Frequency'] = 11
# print(result.to_markdown())
r23 = result

In [192]:
compound = ['myo-Inositol',
            'Inositol',
            'Cyclohexitol'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'myo-Inositol'
result['HMDB_ID'] = 'HMDB0000211'
result['Frequency'] = 11
# print(result.to_markdown())
r24 = result

In [193]:
compound = ['Succinic acid',
            'Succinate',
            'Butandisaeure'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Succinic acid'
result['HMDB_ID'] = 'HMDB0000254'
result['Frequency'] = 11
# print(result.to_markdown())
r25 = result

In [194]:
compound = ['L-Tryptophan',
            'Tryptophan'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'L-Tryptophan'
result['HMDB_ID'] = 'HMDB0000929'
result['Frequency'] = 11
# print(result.to_markdown())
r26 = result

In [195]:
compound = ['Acetic acid',
            'Acetate',
            'Ethanoate'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Acetic acid'
result['HMDB_ID'] = 'HMDB0000042'
result['Frequency'] = 10
# print(result.to_markdown())
r27 = result

In [196]:
compound = ['Uridine',
            'Allouridine'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Uridine'
result['HMDB_ID'] = 'HMDB0000296'
result['Frequency'] = 10
# print(result.to_markdown())
r28 = result

In [197]:
compound = ['Myristic acid',
            'Myristate'
            'Myristoate'
           ]
regex = "|".join(compound)
# print(regex)
result = df.loc[df['name'].str.contains(regex, case=False)].copy()
result['Metabolite_Name'] = 'Myristic acid'
result['HMDB_ID'] = 'HMDB0000806'
result['Frequency'] = 10
# print(result.to_markdown())
r29 = result

In [198]:
ll = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16,
      r17, r18, r19, r20, r21, r22, r23, r24, r25, r26, r27, r28, r29
]

rr = pd.concat(ll)

In [201]:
gg = rr.groupby(['Metabolite_Name', 'HMDB_ID', 'Frequency']).agg({'score':sum, 'kegg_ko':list})
gg['kegg_ko'] = gg['kegg_ko'].apply(lambda x: '; '.join(x))

In [206]:
gg = gg.sort_values('Frequency', ascending=False)

In [207]:
gg.to_markdown()

"|                                        |    score | kegg_ko                                                                                                                                                                                                                                                                                                                                                                                                                        |\n|:---------------------------------------|---------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n| ('Glycine', 'HMDB0000123', 24)         | 3.65349  | K